In [1]:
import numpy as np
import json
import pandas as pd


from matrix_class import ProteinMatrix
from matrix_class import SubMatrix
from cluster_class import AllClusters
from degreelist_class import DegreeList


from connected_components_utils import *
from analysis_utils import *

from func_e.FUNC_E import FUNC_E # import the class
import func_e.vocabs.all as vocabs

In [2]:
dream3_cluster_dict_file = "../data/results/DREAM-3-cc/d3_5_100.json-cluster.json"
dict_of_clusters = {}
# # convert actual cluster file to a dictionary!!
with open(dream3_cluster_dict_file,"r") as cluster_dict_file:
    dict_of_clusters = json.load(cluster_dict_file)

In [3]:
matrix, clusters, degreelist = initialize_matrix_clusters_degreelist("../data/networks/DREAM_files/dream_3.txt", "../data/results/DREAM-3-cc/d3_5_100.json-cluster.json")

In [4]:
genomic_background_filepath = '../data/testing_data/protein_list.txt'
all_go_terms = vocabs.getTerms(['GO'])
term_mapping_filepath = 'term_mapping.txt'
create_term_mapping_list('../data/go-results/dream_3_go_results.tsv', term_mapping_filepath)

In [5]:
og_clusters_querylist_path = 'original_querylist.txt'
clusters.print_querylist_of_clusters_to_file([*range(clusters.get_num_clusters())]
, og_clusters_querylist_path)


original_fe = FUNC_E()
original_fe.importFiles({
    'background': genomic_background_filepath, 
    'terms2features': term_mapping_filepath })
original_fe.setTerms(all_go_terms)
original_fe.setEnrichmentSettings({'ecut': 0.01})

original_fe.importFiles({'query': og_clusters_querylist_path })
original_fe.run(cluster=False)

/Users/charlotte/miniconda3/envs/diamonds/lib/python3.10/site-packages/func_e/FUNC_E.py:279: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  modResults = modResults.append({
/Users/charlotte/miniconda3/envs/diamonds/lib/python3.10/site-packages/func_e/FUNC_E.py:279: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  modResults = modResults.append({
/Users/charlotte/miniconda3/envs/diamonds/lib/python3.10/site-packages/func_e/FUNC_E.py:279: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  modResults = modResults.append({
/Users/charlotte/miniconda3/envs/diamonds/lib/python3.10/site-packages/func_e/FUNC_E.py:279: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat 

In [6]:
original_df = original_fe.enrichment[['Module', 'Term', 'Fishers_pvalue']].copy()
original_df['Module'] = original_df['Module'] + ' ' + original_df['Term']
original_df.drop('Term', axis=1, inplace=True)

### Now: will create different sets of proteins to add back etc

#### take 1: 

step 1: find qualifying proteins and clusters with very strict cutoff, and initialize clusters

In [7]:
qualifying_clusters_1, qualifying_proteins_1 = find_clusters_and_proteins_together(matrix, clusters, degreelist, cluster_ratio=.90, min_components_that_protein_connects=3)

In [10]:
print(qualifying_clusters_1)
print(qualifying_proteins_1)

[25, 152, 161, 185]
{25: ['AKT1', 'SRC'], 152: ['PPM1A', 'PPP2CA', 'CDK2', 'GSK3B', 'MAPK3', 'CSNK2A1', 'PRKACA', 'SRC'], 161: ['SNCA', 'LRP6', 'RPS6KA1', 'NFE2L2', 'ESR1', 'GRB2', 'PLK1', 'STAT3', 'MAPK8', 'ABL1', 'MAPK14', 'GSK3B', 'TP53', 'AKT1', 'CSNK2A1', 'CDK1', 'PRKCA'], 185: ['TSC1', 'IKBKB']}


In [31]:
# new_clusters_1 = AllClusters(protein_to_cluster_dict=dict_of_clusters)
# print(new_clusters_1.get_cluster_proteins(152))
# print(clusters.get_cluster_proteins(152))


# print("")
# new_clusters_1 = modified_clusters(new_clusters_1, qualifying_proteins_1)
# print(new_clusters_1.get_cluster_proteins(152))
#TODO: ERRORRRRR original clusters get their cluster proteins updated as well. 


In [35]:
# from analysis_utils import print_both_querylists_to_files
# from analysis_utils import get_initialized_fe

step 2: run functional enrichment
    a. create og and new querylists
    b. create a comparison and new functional enrichment with querylists as input
    c. run both functional enrichments

In [12]:
original_clusters_querylist_1 = 'og_querylist_1.txt'
updated_clusters_querylist_1 = 'new_querylist_1.txt'

In [51]:
# original:
print_querylist_of_clusters_to_file(clusters, qualifying_clusters_1,original_clusters_querylist_1)

# clusters with added proteins:
print_querylist_of_clusters_to_file(clusters, qualifying_clusters_1, updated_clusters_querylist_1, qualifying_proteins_1)

In [14]:
original_fe_1 = get_initialized_fe(genomic_background_filepath, term_mapping_filepath, termlist = all_go_terms)

updated_fe_1 = get_initialized_fe(genomic_background_filepath, term_mapping_filepath, termlist = all_go_terms)


In [15]:
original_fe_1.importFiles({'query': original_clusters_querylist_1 })
original_fe_1.run(cluster=False)

updated_fe_1.importFiles({'query': updated_clusters_querylist_1 })
updated_fe_1.run(cluster=False)

/Users/charlotte/miniconda3/envs/diamonds/lib/python3.10/site-packages/func_e/FUNC_E.py:279: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  modResults = modResults.append({
/Users/charlotte/miniconda3/envs/diamonds/lib/python3.10/site-packages/func_e/FUNC_E.py:279: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  modResults = modResults.append({
/Users/charlotte/miniconda3/envs/diamonds/lib/python3.10/site-packages/func_e/FUNC_E.py:279: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  modResults = modResults.append({
/Users/charlotte/miniconda3/envs/diamonds/lib/python3.10/site-packages/func_e/FUNC_E.py:279: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat 

KeyboardInterrupt: 

step 3: from functional enrichment, assemble data/p-values
    a. get original and updated pvals
    b. combine in a single dataframe

In [ ]:
original_df_1 = original_fe_1.enrichment[['Module', 'Term', 'Fishers_pvalue']].copy()
original_df_1['Module'] = original_df_1['Module'] + ' ' + original_df_1['Term']
original_df_1.drop('Term', axis=1, inplace=True)

In [ ]:
original_df_1

In [ ]:
updated_df_1 = updated_fe_1.enrichment[['Module', 'Term', 'Fishers_pvalue']].copy()

updated_df_1['Module'] = updated_df_1['Module'] + ' ' + updated_df_1['Term']
updated_df_1.drop('Term', axis=1, inplace=True)
updated_df_1.rename(columns = {'Fishers_pvalue':'Updated_Fishers_pvalue'}, inplace = True)

In [ ]:
updated_df_1

In [ ]:
results_df_1 = pd.merge(original_df_1, updated_df_1, on=['Module'], how='outer')
results_df_1

,Module,Fishers_pvalue,Updated_Fishers_pvalue
0,cluster_161 GO:0030175,0.002023,0.002023
1,cluster_161 GO:0032465,0.007341,0.007341
2,cluster_185 GO:0016241,0.007339,0.007339
3,cluster_185 GO:0016485,0.005832,0.005832
4,cluster_185 GO:0030216,0.004486,0.004486
5,cluster_185 GO:0055038,0.007339,0.007339
6,cluster_185 GO:0070507,0.002297,0.002297
7,cluster_185 GO:0072659,0.001876,0.001876
8,cluster_25 GO:0007346,0.007487,0.007487
9,cluster_25 GO:0032728,0.005714,0.005714


In [ ]:
print(f"ORIGINAL CLUSTERS TABLE:\n{original_df}")
print(f"UPDATED CLUSTERS TABLE:\n{updated_df}")

step 4: display results in a graph

In [ ]:
print(results_df)

In [ ]:
results_df.to_csv("results.txt", sep='\t')


In [ ]:
import matplotlib.pyplot as plt

In [ ]:
results_df.plot(x="Module", y=["Fishers_pvalue", "Updated_Fishers_pvalue"], kind="bar")

In [ ]:
changes_df = (results_df.copy()).dropna(axis=0)

In [ ]:
changes_df.plot(x="Module", y=["Fishers_pvalue", "Updated_Fishers_pvalue"], kind="bar")

this graph ^ is interesting bc it shows that p-values increase (less significant). this shows that maybe we are adding proteins to the wrong clusters. it will be interesting to go back and tweak the functions and see how adding different things affects everything